# reconaissance des chifres manuscrits en utilisant un reseau de neurons formé de plusieurs couches cachées

In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('boston_house_prices.csv')

In [3]:
df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [4]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [5]:
df = np.array(df)

In [6]:
print(type(df))

<class 'numpy.ndarray'>


In [7]:
df.shape

(506, 14)

In [8]:
labels = df[:,df.shape[1]-1]

In [9]:
labels.shape

(506,)

In [10]:
df = df[:,0:df.shape[1]-1]

In [11]:
df.shape

(506, 13)

In [12]:
X_train = df.T
Y_train = labels
print('x train shape est {}\n Y train shape est {}'.format(X_train.shape,Y_train.shape))

x train shape est (13, 506)
 Y train shape est (506,)


In [13]:
X_train = (X_train - np.min(X_train))/np.max(X_train)

In [14]:
print(X_train.max())
print(X_train.min())

1.0
0.0


In [15]:
print(Y_train[0:10])

[24.  21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9]


In [16]:
# def One_Hot(Y) :
#     A = np.zeros((Y.shape[0],np.max(Y)+1))
#     A[range(Y.shape[0]),Y] = 1
#     return A.T

In [17]:
# Y = One_Hot(Y_train)
# Y

In [18]:
# np.random.seed(seed = 30)

In [19]:
def Init_poids(N_N_L_list) :
    # N_N_L_list : est une liste des nombres de neuds de chaque couche
    # par exemple N_N_L_list = [784,15,20,10] cad couche1 contient 784 noeuds
    # couche2 contient 15 noeuds couche3 contient 20 noeuds la derniere couche 
    # contient 10 noueds
    
    nb_layers = len(N_N_L_list)
    param = {}
    
    for i in range(1, nb_layers) :
        param['theta'+str(i)] = np.random.rand(N_N_L_list[i],N_N_L_list[i-1]) - 0.5
        param['b'+str(i)] = np.random.rand(N_N_L_list[i],1) - 0.5
    
    return param

In [20]:
# param = Init_poids([8,7,5,4,3])
# for k,v in param.items() :
#     print(k,v.shape)

In [21]:
# def softmax(X):
#     return np.exp(X)/sum(np.exp(X))

In [22]:
def Relu(X) :
    return np.where(X>=0,X,0) 

In [23]:
# Y1 = Relu(np.array([-1,3]))
# Y1

In [24]:
def Rel_prime(X):
    return np.where(X<=0,0,1)

In [25]:
def coefficient_determination(y,pred) :
    u =((y-pred)**2).sum()
    v = ((y-y.mean())**2).sum()
    return 1 -u/v

In [26]:
def Forward_propagation(param,X):
    nbr_layer = len(param) // 2 + 1
    outp = {'Z1' : X, 'A1' : X}
    for i in range(2,nbr_layer) :
        outp['Z'+str(i)] = np.dot(param['theta'+str(i-1)],outp['A'+str(i-1)])+param['b'+str(i-1)]
        outp['A'+str(i)] = Relu(outp['Z'+str(i)])
    outp['Z'+str(nbr_layer)] = np.dot(param['theta'+str(nbr_layer-1)],outp['A'+str(nbr_layer-1)])+param['b'+str(nbr_layer-1)]
    outp['A'+str(nbr_layer)] = outp['Z'+str(nbr_layer)]
    return outp

In [27]:
# X = np.array([[1,2],[3,4],[2,3],[5,2],[3,9],[6,3],[1,0],[3,9]])
# outp = Forward_propagation(param,X)
# for k,v in outp.items() :
#     print(k,v.shape)

In [28]:
def Backward_propagation(param,outp, X, Y):
    
    nbr_layer = len(param) // 2 + 1
    m = Y.shape[0]
#     Y_enc = One_Hot(Y)
    #Y_enc = np.array([[0,1],[1,0],[0,1]])
    grad = {}
    

    delta = 2 * (outp['A'+str(nbr_layer)] -Y)
    for i in reversed(range(2,nbr_layer)) :
        grad['dtheta' + str(i)] = 1/m * np.dot(delta, outp['A'+str(i)].T)
        grad['db' + str(i)] = 1/m * np.sum(delta, axis=1, keepdims=True)
        delta = np.dot(param['theta'+str(i)].T, delta) * Rel_prime(outp['Z'+str(i)])
    grad['dtheta' + str(1)] = 1/m * np.dot(delta, outp['A'+str(1)].T)
    grad['db' + str(1)] = 1/m * np.sum(delta, axis=1, keepdims=True)
    
    return grad

In [29]:
# Y = np.array([0,2])
# grad = Backward_propagation(param,outp, X, Y)
# for k,v in grad.items() :
#     print(k,v.shape)

In [30]:
def pred(A) :
    return A

In [31]:
# def precision(Y_true,Y_pred) :
#     return np.sum(Y_true == Y_pred)/Y_true.shape[0]

In [32]:
def update_param(param,grad,alpha) :
    nbr_layer = len(param) // 2 + 1
    
    for i in range(1,nbr_layer) :
        param['theta'+str(i)] -= alpha * grad['dtheta'+str(i)]
        param['b'+str(i)] -= alpha * grad['db'+str(i)]
    return param

In [52]:
def grad_descent(X, Y, list_neaurons, alpha, nbitr) :
    
    param = Init_poids(list_neaurons)
    nbr_layer = len(param) // 2 + 1
    for i in range(nbitr) :
        outp = Forward_propagation(param,X)
        grad = Backward_propagation(param,outp, X, Y)
        
        param = update_param(param,grad,alpha)
        
        Y_pred = pred(outp['A'+str(nbr_layer)])
        prec = coefficient_determination(Y,Y_pred)
        
        if i%10000 == 0 :
            print('precision a l iteration {} = {}'
              .format(i,prec))
    return param

In [57]:
param = grad_descent(X_train, Y_train, [13,10,10,1], 0.005, 20001) 

precision a l iteration 0 = -6.23943186867752
precision a l iteration 10000 = 0.4591787361005921
precision a l iteration 20000 = 0.503393415579487
